# Subsample Polygons

### Here we will cut up the drawn polygons into equal sized hexagons

In [1]:
import geopandas as gpd

/home/simonhans/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
plots_path = '../data/polygons/RegressionRidge_smol.geojson'

plots = gpd.read_file(plots_path)

In [3]:
plots.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
plots = plots.to_crs(26910)   # UTM for meter-based subdivision

In [5]:
import numpy as np
from shapely.geometry import Polygon, MultiPolygon

def make_hex(center_x, center_y, r):
    '''Flat-top hexagon with circumradius r.'''
    angles = np.deg2rad([0, 60, 120, 180, 240, 300])
    return Polygon([
        (center_x + r * np.cos(a), center_y + r * np.sin(a))
        for a in angles
    ])

def clean_geom(g, min_area):
    '''Fix invalids, remove slivers, collapse multipolygons.'''
    if g.is_empty:
        return None

    # Fix invalid geometries (self-crossing, slivers, etc.)
    g = g.buffer(0)

    if g.is_empty:
        return None

    # If multipolygon, take largest solid piece
    if isinstance(g, MultiPolygon):
        g = max(g.geoms, key=lambda x: x.area)

    # Final sliver removal
    if g.area < min_area:
        return None

    return g


def subdivide_hexagonal(poly, min_area_m2):
    '''
    Subdivide a polygon into equal-area hexagons (flat top) with no gaps.
    Includes sliver removal & geometry cleanup.
    '''
    # Area of flat-top hex: A = (3√3 / 2) * r²
    r = np.sqrt((2 * min_area_m2) / (3 * np.sqrt(3)))

    # Hex spacing (flat-top geometry)
    horiz = 1.5 * r             # center-to-center horizontally
    vert  = np.sqrt(3) * r      # center-to-center vertically

    minx, miny, maxx, maxy = poly.bounds

    hexes = []
    y = miny
    row = 0

    while y <= maxy + vert:
        # offset every other row
        x_start = minx + (0.75 * r if row % 2 else 0)
        
        x = x_start
        while x <= maxx + horiz:
            h = make_hex(x, y, r)
            clipped = poly.intersection(h)

            # Clean + sliver removal injected HERE
            geom = clean_geom(clipped, min_area_m2 * 0.25)

            if geom is not None:
                hexes.append(geom)

            x += horiz
        
        y += vert
        row += 1

    return hexes


In [6]:
MIN_AREA = 1000  # m²

subplots = []

for idx, row in plots.iterrows():
    poly = row.geometry
    
    
    parent = row.get('Name', f'plot_{idx}')

    parts = subdivide_hexagonal(poly, MIN_AREA)

    n = 0
    for geom in parts:
        
        
        subplots.append({
            'plot_id': parent,
            'geometry': geom
        })
        n += 1

subplots_gdf = gpd.GeoDataFrame(subplots, crs=plots.crs)


In [7]:
sub_gdf = gpd.GeoDataFrame(subplots_gdf, crs=plots.crs)


In [8]:
sub_gdf['plot_id'] = sub_gdf.index

In [9]:
sub_gdf = sub_gdf.to_crs(4326)

In [10]:
sub_gdf

,plot_id,geometry
0,0,"POLYGON ((-119.78208 45.87245, -119.78215 45.8..."
1,1,"POLYGON ((-119.78179 45.87245, -119.78167 45.8..."
2,2,"POLYGON ((-119.78139 45.87246, -119.78129 45.8..."
3,3,"POLYGON ((-119.78299 45.87287, -119.78300 45.8..."
4,4,"POLYGON ((-119.78263 45.87269, -119.78269 45.8..."
...,...,...
3593,3593,"POLYGON ((-119.77610 45.87519, -119.77626 45.8..."
3594,3594,"POLYGON ((-119.77718 45.87568, -119.77718 45.8..."
3595,3595,"POLYGON ((-119.77685 45.87573, -119.77680 45.8..."
3596,3596,"POLYGON ((-119.77652 45.87579, -119.77642 45.8..."


In [11]:
sub_gdf.to_pickle('../data/polygons/RegressionRidge_smol_smol.pkl')